In [3]:
import pandas as pd
import numpy as np
import csv
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchtext
import numpy as np
import matplotlib.pyplot as plt

In [12]:
import torchtext

text_field = torchtext.data.Field(sequential=True,      # text sequence
                                  tokenize=lambda x: x, # because are building a character-RNN
                                  include_lengths=True, # to track the length of sequences, for batching
                                  batch_first=True,
                                  use_vocab=True)       # to turn each character into an integer index
i = 0
for line in open('sp3.txt'):
        #print(line)
        i = i+1
        if i>=10:
            break
fields = [('line', text_field)]
dataset = torchtext.data.TabularDataset("sp3.txt", # name of the file
                                        "csv",  # fields are separated by a tab
                                        fields)
split = [0.6, 0.2,0.2]
train, valid, test =  dataset.split(split, stratified=False, strata_field=None, random_state=None)
text_field.build_vocab(train)
# print(text_field.vocab.stoi)
# print(text_field.vocab.itos)
train_iter = torchtext.data.BucketIterator(train,
                                           batch_size=32,
                                           sort_key=lambda x: len(x.line), # to minimize padding
                                           sort_within_batch=True,        # sort within each batch
                                           repeat=False)                   # repeat the iterator for multiple epochs
for i, batch in enumerate(train_iter):
    if i >= 10:
        break
    lines = batch.line[0]
    print(lines)

tensor([[27,  5,  4,  7, 13, 30,  5, 26, 11,  2,  4,  2, 17, 10,  8,  8,  6,  8,
         18,  2],
        [34,  4,  7,  2, 73, 13, 30,  2, 16,  6,  7, 26,  5,  2, 23,  7,  6, 21,
         18,  2],
        [34,  4,  7,  2, 10,  7,  2, 31,  9,  3,  3, 12, 20,  9,  4, 10,  8, 13,
         44,  6],
        [37,  3,  7,  7, 14, 13, 60, 53,  9,  2,  6,  9, 18,  2, 38,  3, 14,  2,
         15,  9],
        [37, 14, 12,  3, 13, 27,  9,  6,  6,  5,  2, 10,  5,  2, 27,  5,  4,  7,
         18,  2],
        [27, 20,  9,  8,  6,  3, 16,  3,  8, 13, 31,  9,  4,  5,  2, 23,  3, 14,
         35,  2],
        [37, 14, 12,  3,  2, 27, 20,  9, 21,  4,  8,  5, 56, 13, 41,  3, 48, 14,
          3, 11],
        [45,  6, 29,  2, 45,  3, 10,  7,  3,  8, 13, 39, 23,  4, 14,  2, 50,  8,
         10, 20],
        [28,  9,  3, 25, 13, 39,  9, 18,  2, 38,  3, 12, 12,  6,  2,  5,  9,  3,
          8,  3],
        [28,  4,  8,  5, 17,  4,  7, 13, 39,  9,  2, 17, 14,  2, 19,  6,  9,  9,
          9, 16],
        [3

          9,  4, 12, 12,  2, 24,  4, 11, 11,  1]])
tensor([[58,  3,  8,  6,  7, 10, 20,  4, 13, 44,  6,  6,  6,  6, 22,  2,  2],
        [34,  8, 18,  2, 42,  4,  8,  8, 10, 11,  6,  7, 13, 47,  6,  6, 23],
        [37, 14, 12,  3, 13, 41,  6, 15,  2, 16, 10, 16,  7, 26,  5, 35,  2],
        [28,  4,  8,  5, 17,  4,  7, 13, 28,  6, 17,  3,  2, 26,  3,  8,  3],
        [34,  4,  7, 13, 31,  3,  2,  4,  8,  3,  2, 44, 36, 34, 40, 47, 36],
        [46,  9,  6,  5,  6, 19,  8,  4, 24,  9,  3,  8, 13, 39, 23,  4, 14],
        [27,  5,  4,  7, 13, 30,  2, 19, 15,  3, 11, 11, 18, 18, 18,  2,  2],
        [27,  9,  4,  8,  6,  7, 13, 30, 11,  2, 10,  5,  2,  5,  9,  3, 17],
        [40,  8, 10,  4,  7, 13, 28,  6, 17,  3,  2,  6,  7,  2,  6, 15,  5],
        [47,  4, 16, 14,  2, 34,  3, 17, 29,  3,  8, 13, 41,  3, 11, 22,  2],
        [37, 14, 12,  3, 13, 34,  8, 18,  2, 42,  4,  8,  8, 10, 11,  6,  7],
        [45, 15, 11, 11,  3, 12, 12,  2, 28,  8,  6, 21,  3, 13, 41,  3, 11],
        [33, 

          2,  2,  1]])
tensor([[27,  9,  4,  8,  6,  7, 13, 39,  9,  2, 17, 14,  2, 19,  6, 11,  9],
        [40, 15,  5,  5,  3,  8, 11, 13, 39, 21, 22,  2, 43,  4, 16, 35,  2],
        [40, 15,  5,  5,  3,  8, 11, 13, 39, 23,  4, 14, 22,  2, 38,  3, 14],
        [40, 15,  5,  5,  3,  8, 11, 13, 39,  9,  2, 17, 14,  2, 42,  6, 16],
        [47, 10,  4,  7,  3, 13, 33,  9,  4,  5, 26, 11,  2,  7, 10, 20,  3],
        [34, 14, 11,  5,  3,  8, 10,  6,  7, 13, 36, 12,  8, 10, 19,  9,  5],
        [27,  5,  4,  7, 13, 34, 11, 18,  2, 28,  8,  4, 29,  5,  8,  3,  3],
        [51,  8,  6,  5,  9, 19,  4,  8, 13, 53,  9,  2,  9, 15,  9, 35,  2],
        [27,  5,  3, 24,  9,  3,  7, 13, 40, 15,  5,  5,  3,  8, 11, 35,  2],
        [34,  8, 18,  2, 38,  4,  7, 23,  3, 14, 13, 37, 14, 12,  3, 18,  2],
        [40, 15,  5,  5,  3,  8, 11, 13, 33,  9,  3,  2, 21,  9,  6, 35,  2],
        [27,  5,  4,  7, 13, 33,  9,  4,  5, 26, 11,  2, 21,  3, 10,  8, 16],
        [28,  4,  8,  5, 17,  4,  7, 13, 

In [15]:
##a naive model following the lecture notes with 1 hidden layer - GRU 
class CartmanLSTM(nn.Module):
    def __init__(self, vocab_size, hidden_size, n_layers=1):
        super(CartmanLSTM, self).__init__()
        self.vocab_size = vocab_size
        self.hidden_size = hidden_size
        self.n_layer = n_layers
        
        self.ident = torch.eye(vocab_size)
        
        self.rnn = nn.LSTM(vocab_size, hidden_size, n_layers, batch_first = True)
        
        self.decoder = nn.Linear(hidden_size, vocab_size)
    
    def forward(self, x):
        
        
        #set intiial hidden state and cell state
        h0 = torch.zeros(1, x.size(0), self.hidden_size)
        c0 = torch.zeros(1, x.size(0), self.hidden_size)
        
        #forward propogate
        out, _ = self.rnn(x, (h0, c0))
        
        #passs output of last time step to classifier
        out = self.fc(out[:, -1, :])
        
        return out
        
    def init_hidden(self):
        return torch.zeros(self.n_layers, 1, self.hidden_size)

In [16]:
def evaluate(model, prime_str='I', predict_len=300, temperature=0.8):
    
    hidden = model.init_hidden()
    prime_input = text_to_tensor(prime_str)
    predicted = prime_str
    
    # Use priming string to "build up" hidden state
    for p in range(len(prime_str) - 1):
        _, hidden = model(prime_input[p], hidden)
    inp = prime_input[-1]
    
    for p in range(predict_len):
        output, hidden = model(inp, hidden)
        
        # Sample from the network as a multinomial distribution
        output_dist = output.data.view(-1).div(temperature).exp()
        top_i = int(torch.multinomial(output_dist, 1)[0])
        # Add predicted character to string and use as next input
        predicted_char = vocab_itos[top_i]
        predicted += predicted_char
        inp = text_to_tensor(predicted_char)

    return predicted

def train(model, num_iters=2000, lr=0.004):
    
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    
    for it in range(num_iters):
        # get training set
        inp, target = random_training_set()
        # cleanup
        optimizer.zero_grad()
        # forward pass
        hidden = model.init_hidden()
        output, _ = model(inp, hidden)
        loss = criterion(output, target)
        # backward pass
        loss.backward()
        optimizer.step()

        if it % 200 == 199:
            print("[Iter %d] Loss %f" % (it+1, float(loss)))
            print("    " + evaluate(model, ' ', 300))


In [18]:
model = CartmanLSTM(len(vocab), 5000)

NameError: name 'vocab' is not defined